# Create Heat Maps for Red Light Camera Tickets
* Daily Average per location
* Average per 1,000 vehicles per location

### Set up dependencies

In [1]:
# Import dependencies and key
import pandas as pd
import gmaps
from IPython.display import Image

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

# address file to read (generated from 'ChicagoTrafficCount.ipynb')
csv_file = "Chicago_Count_AADT_by_Location.csv"

### Read ticket data

In [2]:
df = pd.read_csv(csv_file)
print(f"Number of locations: {df.shape[0]}")
df.head()

Number of locations: 465


,Unnamed: 0,address,Total,First Ticket Date,Last Ticket Date,Days,Daily Avg,Lat,Lng,zipcode,Traffic Loc Found,Radius,Traffic Address,Date_count,Total Volume,Vol by Direction,Ticket per 1000 AADT
0,0,1 E 63RD ST,12038,3/3/2014,4/27/2018,1517,7.935399,41.780047,-87.625323,60637.0,3.0,500.0,120 East 63rd St,2006-03-28T08:00:00.000Z,12400.0,East Bound: 7800 / West Bound: 4600,1.279903
1,1,1 E 75TH STREET,15159,3/1/2014,4/27/2018,1519,9.979592,41.758293,-87.616316,60619.0,5.0,500.0,504 East 75th St,2006-03-28T08:00:00.000Z,16100.0,East Bound: 8500 / West Bound: 7600,1.239701
2,2,1 E 79TH STREET,38135,3/1/2014,4/27/2018,1519,25.105332,41.750619,-87.624256,60619.0,2.0,500.0,20 East 79th St,2006-03-28T08:00:00.000Z,20700.0,East Bound: 11300 / West Bound: 9400,2.425636
3,3,1 N ASHLAND AVE,7229,3/1/2014,4/27/2018,1519,4.759052,41.899593,-87.667609,60622.0,7.0,500.0,755 North Ashland Ave,2006-10-26T07:00:00.000Z,30500.0,North Bound: 15800 / South Bound: 14700,0.312069
4,4,1 N HALSTED STREET,5696,3/1/2014,4/27/2018,1519,3.749835,41.882169,-87.646977,60661.0,14.0,500.0,159 North Halsted St,2006-10-02T07:00:00.000Z,20000.0,North Bound: 10200 / South Bound: 9800,0.374984


### Daily Avg Heat Map

In [3]:
# Heat Map Data to plot
locations = df[["Lat", "Lng"]]
weights = df['Daily Avg']

# Top 5 locations to show as markers
df_top5 = df.nlargest(5,'Daily Avg')[['address', 'Daily Avg', 'Lat', 'Lng']]
df_top5['Daily Avg'] = df_top5['Daily Avg'].round(0)

# Using the template add marks to the heatmap
info_box_template = """
<dl>
<dt>Loc</dt><dd>{address}</dd>
<dt>Daily Tix #</dt><dd>{Daily Avg}</dd>
</dl>
"""

# Store the DataFrame Row
camera_info = [info_box_template.format(**row) for index, row in df_top5.iterrows()]
loc_top5 = df_top5[["Lat", "Lng"]]

In [4]:
# Plot Map
fig = gmaps.figure(
    layout={
        'width': '500px',
        'height': '800px'
    }
)

# Create and add heat layer
heat_layer = gmaps.heatmap_layer(
    locations,
    weights=weights, 
    dissipating=False,
    max_intensity=max(weights),
    point_radius=0.02)
fig.add_layer(heat_layer)

# Create and add marker layer
markers = gmaps.marker_layer(
    loc_top5,
    info_box_content=camera_info
)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='800px', width='500px'))

![title](DailyAvgHeatMap.png)

### Average per 1,000 Vehicles Map

In [5]:
df_aadt = df.dropna()

# Heat Map Data to plot
locations_aadt = df_aadt[["Lat", "Lng"]]
weights_aadt = df_aadt['Ticket per 1000 AADT']

# Top 5 locations to show as markers
df_aadt_top5 = df_aadt.nlargest(5,'Ticket per 1000 AADT')[[
    'address', 'Ticket per 1000 AADT', 'Lat', 'Lng']]
df_aadt_top5['Ticket per 1000 AADT'] = \
    df_aadt_top5['Ticket per 1000 AADT'].round(1)

# Using the template add marks to the heatmap
info_box_template = """
<dl>
<dt>Loc</dt><dd>{address}</dd>
<dt>Tix per 1,000 Vehicle #</dt><dd>{Ticket per 1000 AADT}</dd>
</dl>
"""

# Store the DataFrame Row
camera_info_aadt = [info_box_template.format(**row) for index, row in df_aadt_top5.iterrows()]
loc_aadt_top5 = df_aadt_top5[["Lat", "Lng"]]

In [6]:
# Plot Map
fig2 = gmaps.figure(
    layout={
        'width': '500px',
        'height': '800px'
    }
)

# Create and add heat layer
heat_layer = gmaps.heatmap_layer(
    locations_aadt,
    weights=weights_aadt, 
    dissipating=False,
    max_intensity=max(weights_aadt),
    point_radius=0.02)
fig2.add_layer(heat_layer)

# Create and add marker layer
markers = gmaps.marker_layer(
    loc_aadt_top5,
    info_box_content=camera_info_aadt
)
fig2.add_layer(markers)

# Display figure
fig2

Figure(layout=FigureLayout(height='800px', width='500px'))

![title](AvgPerVehicleHeatMap.png)